# Úloha klasifikácie textu

V tomto module začneme jednoduchou úlohou klasifikácie textu založenou na dátovej sade **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: budeme klasifikovať nadpisy správ do jednej zo 4 kategórií: Svet, Šport, Biznis a Veda/Technológie.

## Dátová sada

Na načítanie dátovej sady použijeme API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Teraz môžeme pristupovať k tréningovej a testovacej časti dátovej sady pomocou `dataset['train']` a `dataset['test']`:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Poďme vytlačiť prvých 10 nových titulkov z nášho datasetu:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vektorizácia textu

Teraz musíme previesť text na **čísla**, ktoré môžu byť reprezentované ako tenzory. Ak chceme reprezentáciu na úrovni slov, musíme urobiť dve veci:

* Použiť **tokenizér** na rozdelenie textu na **tokeny**.
* Vytvoriť **slovník** týchto tokenov.

### Obmedzenie veľkosti slovníka

V príklade s datasetom AG News je veľkosť slovníka pomerne veľká, viac ako 100 tisíc slov. Vo všeobecnosti nepotrebujeme slová, ktoré sa v texte vyskytujú zriedkavo — len niekoľko viet ich bude obsahovať, a model sa z nich nenaučí. Preto má zmysel obmedziť veľkosť slovníka na menší počet tým, že odovzdáme argument konštruktoru vektorizéra:

Obe tieto kroky je možné zvládnuť pomocou vrstvy **TextVectorization**. Poďme vytvoriť objekt vektorizéra a potom zavolať metódu `adapt`, aby sme prešli všetok text a vytvorili slovník:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Poznámka**: Používame iba podmnožinu celého datasetu na vytvorenie slovníka. Robíme to preto, aby sme urýchlili čas vykonávania a nemuseli ste dlho čakať. Avšak, týmto podstupujeme riziko, že niektoré slová z celého datasetu nebudú zahrnuté do slovníka a budú ignorované počas tréningu. Použitie celej veľkosti slovníka a prebehnutie celého datasetu počas `adapt` by malo zvýšiť konečnú presnosť, ale nie výrazne.

Teraz môžeme pristúpiť k samotnému slovníku:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Pomocou vektorizátora môžeme ľahko zakódovať akýkoľvek text do množiny čísel:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Reprezentácia textu pomocou metódy Bag-of-words

Keďže slová nesú význam, niekedy dokážeme pochopiť význam textu len na základe jednotlivých slov, bez ohľadu na ich poradie vo vete. Napríklad pri klasifikácii správ môžu slová ako *počasie* a *sneh* naznačovať *predpoveď počasia*, zatiaľ čo slová ako *akcie* a *dolár* by mohli poukazovať na *finančné správy*.

**Bag-of-words** (BoW) je najjednoduchšia tradičná metóda reprezentácie textu pomocou vektorov. Každé slovo je priradené k indexu vektora a prvok vektora obsahuje počet výskytov daného slova v konkrétnom dokumente.

![Obrázok znázorňujúci, ako je reprezentácia Bag-of-words uložená v pamäti.](../../../../../translated_images/sk/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Na metódu BoW sa môžete pozerať aj ako na súčet všetkých vektorov s jedným aktívnym prvkom (one-hot-encoded) pre jednotlivé slová v texte.

Nižšie je uvedený príklad, ako vytvoriť reprezentáciu Bag-of-words pomocou knižnice Scikit Learn v jazyku Python:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Môžeme tiež použiť Keras vektorizátor, ktorý sme definovali vyššie, previesť každé číslo slova na one-hot kódovanie a všetky tieto vektory sčítať:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Poznámka**: Môže vás prekvapiť, že výsledok sa líši od predchádzajúceho príkladu. Dôvodom je, že v príklade s Kerasom dĺžka vektora zodpovedá veľkosti slovníka, ktorý bol vytvorený z celého datasetu AG News, zatiaľ čo v príklade so Scikit Learn sme slovník vytvorili na základe ukážkového textu za pochodu.


## Trénovanie klasifikátora BoW

Teraz, keď sme sa naučili, ako vytvoriť reprezentáciu textu pomocou metódy bag-of-words, poďme natrénovať klasifikátor, ktorý ju využíva. Najskôr musíme náš dataset previesť na reprezentáciu bag-of-words. To môžeme dosiahnuť použitím funkcie `map` nasledovným spôsobom:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Teraz definujme jednoduchú klasifikačnú neurónovú sieť, ktorá obsahuje jednu lineárnu vrstvu. Veľkosť vstupu je `vocab_size` a veľkosť výstupu zodpovedá počtu tried (4). Keďže riešime klasifikačnú úlohu, konečná aktivačná funkcia je **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Keďže máme 4 triedy, presnosť nad 80 % je dobrý výsledok.

## Trénovanie klasifikátora ako jednej siete

Keďže vektorizér je tiež vrstva Keras, môžeme definovať sieť, ktorá ho zahŕňa, a trénovať ju od začiatku do konca. Týmto spôsobom nemusíme vektorizovať dataset pomocou `map`, stačí nám pôvodný dataset poslať na vstup siete.

> **Poznámka**: Stále by sme museli aplikovať mapy na náš dataset, aby sme konvertovali polia zo slovníkov (ako `title`, `description` a `label`) na dvojice. Avšak pri načítavaní dát z disku môžeme hneď na začiatku vytvoriť dataset s požadovanou štruktúrou.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigramy, trigramy a n-gramy

Jedným z obmedzení prístupu bag-of-words je, že niektoré slová sú súčasťou viacslovných výrazov. Napríklad slovo 'hot dog' má úplne iný význam ako slová 'hot' a 'dog' v iných kontextoch. Ak by sme slová 'hot' a 'dog' vždy reprezentovali rovnakými vektormi, mohlo by to zmiasť náš model.

Na riešenie tohto problému sa často používajú **n-gramové reprezentácie** v metódach klasifikácie dokumentov, kde frekvencia každého slova, dvojslovného alebo trojslovného výrazu predstavuje užitočný prvok na trénovanie klasifikátorov. Napríklad v bigramových reprezentáciách pridáme do slovníka všetky dvojice slov, okrem pôvodných slov.

Nižšie je uvedený príklad, ako generovať bigramovú reprezentáciu bag-of-words pomocou Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Hlavnou nevýhodou prístupu n-gramov je, že veľkosť slovníka začne rásť extrémne rýchlo. V praxi je potrebné skombinovať reprezentáciu n-gramov s technikou redukcie dimenzií, ako sú *embeddingy*, o ktorých budeme hovoriť v ďalšej jednotke.

Aby sme mohli použiť reprezentáciu n-gramov v našej **AG News** množine údajov, musíme parameter `ngrams` odovzdať do nášho konštruktora `TextVectorization`. Dĺžka bigramového slovníka je **výrazne väčšia**, v našom prípade ide o viac ako 1,3 milióna tokenov! Preto má zmysel obmedziť aj bigramové tokeny na nejaký rozumný počet.

Mohli by sme použiť rovnaký kód ako vyššie na trénovanie klasifikátora, avšak bolo by to veľmi neefektívne z hľadiska pamäte. V ďalšej jednotke budeme trénovať bigramový klasifikátor pomocou embeddingov. Medzitým môžete experimentovať s trénovaním bigramového klasifikátora v tomto notebooku a zistiť, či dokážete dosiahnuť vyššiu presnosť.


## Automatický výpočet BoW vektorov

V príklade vyššie sme BoW vektory počítali ručne sčítaním one-hot kódovaní jednotlivých slov. Avšak, najnovšia verzia TensorFlow nám umožňuje vypočítať BoW vektory automaticky, a to pridaním parametra `output_mode='count'` do konštruktora vektorizéra. Týmto spôsobom je definovanie a trénovanie nášho modelu výrazne jednoduchšie:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frekvencia termínov - inverzná frekvencia dokumentov (TF-IDF)

V reprezentácii BoW sa výskyty slov vážia rovnakou technikou bez ohľadu na samotné slovo. Je však zrejmé, že časté slová ako *a* a *v* sú oveľa menej dôležité pre klasifikáciu než špecializované výrazy. Pri väčšine úloh NLP sú niektoré slová relevantnejšie než iné.

**TF-IDF** znamená **frekvencia termínov - inverzná frekvencia dokumentov**. Ide o variáciu metódy bag-of-words, kde namiesto binárnej hodnoty 0/1, ktorá označuje prítomnosť slova v dokumente, sa používa hodnota s pohyblivou desatinnou čiarkou, ktorá súvisí s frekvenciou výskytu slova v korpuse.

Formálne je váha $w_{ij}$ slova $i$ v dokumente $j$ definovaná ako:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kde
* $tf_{ij}$ je počet výskytov $i$ v $j$, teda hodnota BoW, ktorú sme videli predtým
* $N$ je počet dokumentov v kolekcii
* $df_i$ je počet dokumentov obsahujúcich slovo $i$ v celej kolekcii

Hodnota TF-IDF $w_{ij}$ rastie úmerne počtu výskytov slova v dokumente a je upravená počtom dokumentov v korpuse, ktoré obsahujú dané slovo, čo pomáha zohľadniť fakt, že niektoré slová sa vyskytujú častejšie než iné. Napríklad, ak sa slovo vyskytuje *v každom* dokumente kolekcie, $df_i=N$, a $w_{ij}=0$, a tieto výrazy by boli úplne ignorované.

TF-IDF vektorizáciu textu môžete jednoducho vytvoriť pomocou Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

V Keras môže vrstva `TextVectorization` automaticky vypočítať TF-IDF frekvencie pridaním parametra `output_mode='tf-idf'`. Poďme zopakovať kód, ktorý sme použili vyššie, aby sme zistili, či použitie TF-IDF zvyšuje presnosť:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Záver

Aj keď reprezentácie TF-IDF poskytujú váhové hodnoty pre rôzne slová na základe ich frekvencie, nedokážu zachytiť význam alebo poradie. Ako povedal známy lingvista J. R. Firth v roku 1935: „Úplný význam slova je vždy kontextový a žiadna štúdia významu mimo kontextu nemôže byť braná vážne.“ Neskôr v kurze sa naučíme, ako zachytiť kontextové informácie z textu pomocou jazykového modelovania.



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
